In [19]:
def schoolbook_mul64(a, b):
    """64×64 → 128-bit GF(2) multiplication. Returns integer (not limbs)."""
    a &= (1 << 64) - 1
    b &= (1 << 64) - 1
    low = 0
    high = 0
    for i in range(64):
        if (a >> i) & 1:
            if i == 0:
                low ^= b
            else:
                low ^= b << i
                high ^= b >> (64 - i)
    return low | (high << 64)

def karatsuba_gf2_recursive(a, b):
    """
    Recursively multiply two GF(2) polynomials (represented as Python ints).
    Uses schoolbook_mul64 when both a and b fit in 64 bits.
    """
    # Base case: both operands fit in 64 bits
    if a < (1 << 64) and b < (1 << 64):
        return schoolbook_mul64(a, b)

    # Ensure a is the longer operand (optional optimization)
    if a.bit_length() < b.bit_length():
        a, b = b, a

    # If b is zero, return zero
    if b == 0:
        return 0

    # Split a around its midpoint
    n = a.bit_length()
    m = (n + 1) // 2  # ceiling(n/2)

    # Split: a = a1 * x^m + a0
    a0 = a & ((1 << m) - 1)
    a1 = a >> m

    # Split b similarly (even if b is shorter)
    b0 = b & ((1 << m) - 1)
    b1 = b >> m

    # Three recursive products
    z0 = karatsuba_gf2_recursive(a0, b0)          # a0 * b0
    z2 = karatsuba_gf2_recursive(a1, b1)          # a1 * b1

    # (a0 + a1) and (b0 + b1) → XOR in GF(2)
    a0_a1 = a0 ^ a1
    b0_b1 = b0 ^ b1

    z1 = karatsuba_gf2_recursive(a0_a1, b0_b1)    # (a0+a1)*(b0+b1)
    z1 ^= z0 ^ z2                                 # z1 = z1 - z0 - z2 → XOR in GF(2)

    # Combine: z2 * x^(2m) + z1 * x^m + z0
    return (z2 << (2 * m)) ^ (z1 << m) ^ z0

In [22]:
# Example: two 17669-bit polynomials
BIT_WIDTH = 17669

a = 12354532453      # x^17668 + 12345
b = 12543532453  # sparse polynomial

# Multiply using recursive Karatsuba
product = karatsuba_gf2_recursive(a, b)

print(product)

78822079870137848529
